In [1]:
import random
from bolib.Bo import Bo
from bolib.Dimension import NumericDimension
from bolib.ComputeSpace import ComputeSpace
from bolib.Normalizer import OptimizeFor
import pandas as pd
from IPython.display import clear_output
import numpy as np

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blackbox_dims = 10

In [3]:

def getRating(v, target):    
    return [sum([1 for i in v if i in target])]
    
def getSample(TARGET = None):
    s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]

    if TARGET is not None:
        
        r= getRating(s, TARGET)
        return [s, r]
    return s


TARGET = getSample()

s, r = getSample(TARGET)
pd.DataFrame([[s, r, TARGET]], columns=["guess", "rating", "target"])  

,guess,rating,target
0,"[3, 19, 22, 39, 45, 55, 66, 76, 80, 95]",[3],"[7, 12, 29, 39, 48, 55, 60, 76, 87, 94]"


In [4]:
[ [i*10+j for j in range(10)] for i in range(blackbox_dims) ]
s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]
s

[1, 10, 27, 35, 46, 58, 67, 75, 86, 94]

In [5]:
from bolib.Dimension import DiscreteDimension


numerics = [ DiscreteDimension(elements=[i*10+j for j in range(10)]  ,name=f"p{i}") for i in range(blackbox_dims) ]
ranking = [ NumericDimension(min=0, max=10, name="Ranking", optimize_for=OptimizeFor.MIN) ]



def infer(x, y, bo, cs):  
  cs.add_values(xs = [x], ys=[y])   

  inf = bo.infer()
  denorm = cs.denormalize(inf)
  return [d.value for d in denorm]





In [6]:
for i in numerics:
  print(i._elements)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [7]:
compSpace = ComputeSpace(numerics, ranking)
bo = Bo(compSpace)

last_rating = 0
last_guess = None
best_guess = None
best_rating = None
print(f"We are looking for {TARGET}")

df_cols = ["guess", "rating", "target"]
df = pd.DataFrame([], columns=df_cols)  

for i in range(50):
  last_rating = r[0]
  last_guess = s
  
  
  guess = infer(s,r, bo, compSpace)  
  print("guess", guess)
  r = getRating(guess, TARGET)

  if r[0] == 0:
    guess = getSample()

  tdf = pd.DataFrame([[s, r, TARGET]], columns=df_cols)  
  
  df = pd.concat([df, tdf], axis=0).reset_index(drop=True)
  
  clear_output(wait=True)

  print(df)

  
  #if r[0] == 2 
  """
  if r[0] < 0.15:
    print(f"Found it in {i} steps: {guess}")
    break
  """
  if best_guess is None:
    best_rating = r[0]
    best_guess = guess

  if r[0] > best_rating:
    best_guess = guess
    best_rating = r[0]
  s = guess

print(f"I think it is {best_guess} with a rating of {best_rating}")


  


  



                                      guess rating  \
0   [1, 10, 27, 35, 46, 58, 67, 75, 86, 94]    [0]   
1   [3, 15, 21, 35, 41, 54, 69, 75, 83, 94]    [1]   
2   [4, 17, 20, 35, 40, 52, 69, 75, 82, 94]    [1]   
3   [2, 14, 20, 35, 40, 56, 69, 75, 83, 94]    [1]   
4   [3, 15, 22, 35, 42, 53, 69, 75, 84, 94]    [2]   
5   [4, 15, 21, 35, 41, 55, 69, 75, 81, 94]    [1]   
6   [2, 16, 20, 35, 40, 53, 69, 75, 84, 94]    [1]   
7   [4, 14, 20, 35, 40, 53, 69, 75, 83, 94]    [2]   
8   [3, 17, 21, 35, 41, 55, 69, 75, 83, 94]    [1]   
9   [1, 14, 21, 35, 41, 53, 69, 75, 83, 94]    [1]   
10  [3, 14, 20, 36, 41, 54, 69, 76, 84, 95]    [0]   
11  [7, 18, 24, 30, 49, 50, 62, 73, 87, 90]    [3]   
12  [7, 18, 23, 30, 48, 50, 62, 73, 87, 90]    [0]   
13  [0, 19, 20, 39, 48, 52, 61, 74, 85, 93]    [1]   
14  [0, 19, 20, 39, 47, 51, 62, 74, 85, 93]    [2]   
15  [0, 19, 20, 39, 49, 53, 60, 74, 85, 93]    [0]   
16  [2, 15, 23, 37, 47, 57, 62, 71, 89, 95]    [0]   
17  [1, 14, 27, 38, 47, 57, 

In [8]:
print(f"I think it is \n{best_guess} with a rating of {best_rating}")
print(TARGET)


I think it is 
[7, 18, 23, 30, 48, 50, 62, 73, 87, 90] with a rating of 3
[7, 12, 29, 39, 48, 55, 60, 76, 87, 94]
